<a href="https://colab.research.google.com/github/suryardh/Stella-train-test/blob/main/stella_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!apt update && apt upgrade -y
!pip uninstall pyarrow -y
!pip install pyarrow==15.0.0
!apt install python3.8 python3.8-dev python3.8-distutils libpython3.8-dev
!pip install tensorflow transformers datasets

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
4 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as 

In [8]:
from datasets import load_dataset

ds = load_dataset("Lots-of-LoRAs/task1533_daily_dialog_formal_classification")

In [11]:
from datasets import load_dataset

ds = load_dataset("Lots-of-LoRAs/task1533_daily_dialog_formal_classification")
print(ds.column_names)


{'train': ['input', 'output', 'id'], 'test': ['input', 'output', 'id'], 'valid': ['input', 'output', 'id']}


In [15]:
from collections import defaultdict
import re
from datasets import load_dataset

class SimpleTokenizer:
    def __init__(self):
        self.vocab = {}
        self.inverse_vocab = {}
        self.current_id = 1  # Start from 1, leave 0 for padding or special tokens

    def build_vocab(self, texts):
        word_freq = defaultdict(int)

        # Count word frequencies
        for text in texts:
            words = re.findall(r'\b\w+\b', text.lower())
            for word in words:
                word_freq[word] += 1

        # Build vocab
        for word in word_freq:
            if word not in self.vocab:
                self.vocab[word] = self.current_id
                self.inverse_vocab[self.current_id] = word
                self.current_id += 1

    def encode(self, text):
        # Convert text to integer tokens based on vocab
        words = re.findall(r'\b\w+\b', text.lower())
        return [self.vocab.get(word, 0) for word in words]

    def decode(self, tokens):
        # Convert integer tokens back to text
        return ' '.join([self.inverse_vocab.get(token, '[UNK]') for token in tokens])

    def vocab_size(self):
        return len(self.vocab) + 1  # +1 for padding or special tokens

# Load the dataset
ds = load_dataset("Lots-of-LoRAs/task1533_daily_dialog_formal_classification")

# Extract texts from the relevant column
texts = ds['train']['input']  # Replace 'text' with the actual column name

# Create and use the tokenizer
tokenizer = SimpleTokenizer()
tokenizer.build_vocab(texts)

# Example of encoding and decoding
sample_text = "This is a sample sentence."
encoded = tokenizer.encode(sample_text)
decoded = tokenizer.decode(encoded)

print("Vocabulary Size:", tokenizer.vocab_size())
print("Encoded:", encoded)
print("Decoded:", decoded)


Vocabulary Size: 13024
Encoded: [184, 25, 5, 4582, 6888]
Decoded: this is a sample sentence


In [16]:
import torch
from torch.utils.data import Dataset, DataLoader

# Assuming SimpleTokenizer is already defined and texts are extracted
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokens = self.tokenizer.encode(text)
        return torch.tensor(tokens), torch.tensor(label)

# Load the dataset
ds = load_dataset("Lots-of-LoRAs/task1533_daily_dialog_formal_classification")

# Extract texts and labels
texts = ds['train']['input']  # Replace 'text' with the actual column name
labels = ds['train']['output']  # Replace 'label' with the actual column name

# Tokenizer
tokenizer = SimpleTokenizer()
tokenizer.build_vocab(texts)

# Create Dataset and DataLoader
train_dataset = TextDataset(texts, labels, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [18]:
from sklearn.preprocessing import LabelEncoder

# Initialize label encoder
label_encoder = LabelEncoder()

# Fit and transform labels
encoded_labels = label_encoder.fit_transform(labels)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [19]:
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokens = self.tokenizer.encode(text)
        return torch.tensor(tokens), torch.tensor(label)

# Create Dataset and DataLoader
train_dataset = TextDataset(texts, encoded_labels, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [24]:
# Ekstrak data validasi
valid_texts = ds['valid']['input']  # Ambil data input untuk validasi
valid_labels = ds['valid']['output']  # Ambil data output untuk validasi

# Encode label validasi
encoded_valid_labels = label_encoder.transform(valid_labels)

# Buat dataset dan dataloader untuk validasi
valid_dataset = TextDataset(valid_texts, encoded_valid_labels, tokenizer)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [27]:
import torch
import numpy as np

# Define EarlyStopping class
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = np.inf
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# Define the model, criterion, and optimizer
model = SimpleModel(vocab_size, embed_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Initialize early stopping
early_stopping = EarlyStopping(patience=3)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    for tokens, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(tokens)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_dataloader)

    # Validation
    model.eval()
    total_valid_loss = 0
    with torch.no_grad():
        for tokens, labels in valid_dataloader:
            outputs = model(tokens)
            loss = criterion(outputs, labels)
            total_valid_loss += loss.item()

    avg_valid_loss = total_valid_loss / len(valid_dataloader)

    # Print training and validation losses
    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss}, Valid Loss: {avg_valid_loss}')

    # Check early stopping
    early_stopping(avg_valid_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        # Save the best model
        torch.save(model.state_dict(), 'best_model.pth')
        break

# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

# Evaluate on test data
model.eval()
total_test_loss = 0
with torch.no_grad():
    for tokens, labels in test_dataloader:
        outputs = model(tokens)
        loss = criterion(outputs, labels)
        total_test_loss += loss.item()

avg_test_loss = total_test_loss / len(test_dataloader)
print(f'Test Loss: {avg_test_loss}')


Epoch 1/50, Train Loss: 0.6648402083304621, Valid Loss: 0.6551888465881348
Epoch 2/50, Train Loss: 0.6619510308388741, Valid Loss: 0.6536314308643341
Epoch 3/50, Train Loss: 0.663918383659855, Valid Loss: 0.6539283126592637
Epoch 4/50, Train Loss: 0.6594446595638029, Valid Loss: 0.6542297095060349
Epoch 5/50, Train Loss: 0.6605251950602378, Valid Loss: 0.6497737884521484
Epoch 6/50, Train Loss: 0.6555539185001004, Valid Loss: 0.6492236584424973
Epoch 7/50, Train Loss: 0.6549634302816083, Valid Loss: 0.6466265052556992
Epoch 8/50, Train Loss: 0.6509084047809723, Valid Loss: 0.6436367928981781
Epoch 9/50, Train Loss: 0.6452099311736322, Valid Loss: 0.638955295085907
Epoch 10/50, Train Loss: 0.6355782570377473, Valid Loss: 0.6351173341274261
Epoch 11/50, Train Loss: 0.6284917889102812, Valid Loss: 0.6305531054735184
Epoch 12/50, Train Loss: 0.6173550938406298, Valid Loss: 0.6249002635478973
Epoch 13/50, Train Loss: 0.6056306529429651, Valid Loss: 0.6221257418394088
Epoch 14/50, Train Loss

<ipython-input-27-7eedecb2c3b2>:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.6277572944760322


In [28]:
# Simpan model terbaik
torch.save(model.state_dict(), 'Stella Model.pth')
